In [1]:
# Load packages.
import os
import sys
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import BertModel, BertTokenizer, BertConfig
from transformers import AdamW, get_linear_schedule_with_warmup

import pandas as pd
import numpy as np
import json

import tqdm
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs


import wandb

os.environ['CURL_CA_BUNDLE'] = ''



d:\Anaconda\envs\similarity_engine\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
key = json.load(open('D:\similarity-engine\wandb_config.json'))
wandb.login(key=key['key'])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.wandb.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.wandb.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
wandb: Currently logged in as: simpleparadox. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider set

True

In [3]:
# # Load data
# data = pd.read_parquet("../data/Processed_records.parquet")
# data = data.dropna()
# data.head()


In [4]:
# 
df = pd.read_csv("df_training_full.csv")

In [5]:
# Divide the dataset into training and test sets.
train_set = df.sample(frac=0.9, random_state=42)  # Fixing the seed to 42 to reproducibility.
test_set = df.drop(train_set.index)

In [6]:
len(train_set)

6438

In [7]:
# # Create a dataframe from simpletransformers. To fine-tune a language model, each sample should be a row in a text file.
# # Store the 'metadata_en_processed' column in a text file.
# with open('../data/simpletransformer_lm_train.txt', 'w') as f:
#     for item in train_set['metadata_en_processed']:
#         f.write("%s\n" % item)

# # Store the test set in a text file.
# with open('../data/simpletransformer_lm_test.txt', 'w') as f:
#     for item in test_set['metadata_en_processed']:
#         f.write("%s\n" % item)



## Simpletransformers code for sanity check.

In [8]:
model_args = LanguageModelingArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 20
model_args.dataset_type = "simple"
model_args.wandb_project = "geo.ca"
model_args.evaluate_during_training = False
model_args.train_batch_size = 8
model_args.eval_batch_size = 8


In [9]:
# Print the model arguments.
print(json.dumps(model_args.__dict__, indent=2))
# print(len(model_args.__dict__))

{
  "adafactor_beta1": null,
  "adafactor_clip_threshold": 1.0,
  "adafactor_decay_rate": -0.8,
  "adafactor_eps": [
    1e-30,
    0.001
  ],
  "adafactor_relative_step": true,
  "adafactor_scale_parameter": true,
  "adafactor_warmup_init": true,
  "adam_betas": [
    0.9,
    0.999
  ],
  "adam_epsilon": 1e-08,
  "best_model_dir": "outputs/best_model",
  "cache_dir": "cache_dir/",
  "config": {},
  "cosine_schedule_num_cycles": 0.5,
  "custom_layer_parameters": [],
  "custom_parameter_groups": [],
  "dataloader_num_workers": 0,
  "do_lower_case": false,
  "dynamic_quantize": false,
  "early_stopping_consider_epochs": false,
  "early_stopping_delta": 0,
  "early_stopping_metric": "eval_loss",
  "early_stopping_metric_minimize": true,
  "early_stopping_patience": 3,
  "encoding": null,
  "eval_batch_size": 8,
  "evaluate_during_training": false,
  "evaluate_during_training_silent": true,
  "evaluate_during_training_steps": 2000,
  "evaluate_during_training_verbose": false,
  "evaluate_

In [10]:
# Set path to the language modelling train file.
train_file = "../data/simpletransformer_lm_train.txt"
test_file = "../data/simpletransformer_lm_test.txt"

In [11]:
use_cuda = torch.cuda.is_available()
model = LanguageModelingModel(
    "bert", "bert-base-uncased", args=model_args, use_cuda=use_cuda
)
print("Running on GPU: {}".format(use_cuda))

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another ar

Running on GPU: True


d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
model.model()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [14]:
# Fine tune.
model.train_model(train_file)#, eval_file=test_file)



Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 of 20:   0%|          | 0/20 [00:10<?, ?it/s]d:\Anaconda\envs\similarity_engine\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 1 of 20:   0%|          | 0/20 [01:15<?, ?it/s]


KeyboardInterrupt: 

In [16]:
result = model.eval_model(test_file)
wandb.log(result)

{'eval_loss': 2.0003846917991286, 'perplexity': tensor(7.3919)}

## Pytorch / Huggingface fine-tuning code.

Some hyperparameters to consider when using huggingface.

use fp16 to fine-tune.

use the AdamW optimzer.

scheduler: use linear schedule with warmup

batch_size for everything: 8

early stopping True

early stopping patience: 3

gradient_accumulation_steps=1

learning_rate=4e-05

max_grad_norm=1.0, 

max_seq_length=128

warmup_ratio=0.06

warmup_steps=0

strip_accents=True

handle_chinese_characters=True


In [31]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# from sklearn.metrics.pairwise import cosine_similarity

# # Load the BERT model
# bert_tokenizer = AutoTokenizer.from_pretrained('D:\\similarity-engine\\Jupyter_notebooks\\outputs\\')




Some weights of BertModel were not initialized from the model checkpoint at D:\similarity-engine\Jupyter_notebooks\outputs\ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import sys
sys.path.append("D:\\similarity-engine\\src\\")
from utils.custom_dataset import CustomDataset
from models.custom_model import CustomModel
import pandas as pd
import torch
import wandb

In [17]:
wandb.init(project='geo.ca')
config = wandb.config

Training loss,▄█▃▃▄▄▂▁▁▄
global_step,▁▂▃▃▄▅▆▆▇█
lr,▁▂▃▃▄▅▆▆▇█
Training loss,3.89149
global_step,500
lr,2e-05


In [18]:
config.model_name = 'bert-base-uncased'
config.model_type = 'bert'
config.batch_size = 8
config.epochs = 3
config.lr = 2e-5

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device: ", device)
config.device = device
print(config.device)

Device:  cuda:0
cuda:0


In [5]:
df = pd.read_csv('D:\\similarity-engine\\notebooks\\df_training_full.csv')

# Divide the data into train and test set. 
train_set = df.sample(frac=0.9, random_state=42)  # Fixing the seed to 42 to reproducibility.
test_set = df.drop(train_set.index)


In [21]:
# Initialize the model.
model = CustomModel(config)

# Train the model.
model.train_model(train_set)
print(model.model)

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

Training the model on device:  cuda:0


Epoch 1 of 3:   0%|          | 0/3 [00:29<?, ?it/s]


KeyboardInterrupt: 

## Simpletransformers models.
Fine tuning BERT using LanguageModellingModel and the setting the finetuned model to a RepresentationModel.

In [26]:
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs


In [39]:
# First encode a sentence with the pretrained representation model.

# E.g., BERT pretrained.
# Can be a model from HuggingFace's transformers library.
model_type = 'bert'
model_name = 'bert-base-uncased'
rep_model = RepresentationModel(
        model_type=model_type,
        model_name=model_name,
        use_cuda=True
    )

sentences = ['hello world 1', 'hello world 2']

word_vectors = rep_model.encode_sentences(sentences, combine_strategy=None)

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [40]:
word_vectors.shape
# Take the first embedding of each sentence as the CLS embedding.

(2, 5, 768)

In [106]:
# Now let's fine tune a model.

# Initialize the LM Args.
model_args = LanguageModelingArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 1
model_args.dataset_type = "simple"
# model_args.wandb_project = "geo.ca"
model_args.evaluate_during_training = False
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.config = {"output_hidden_states": True}

In [107]:
# Specify data paths.
train_file = "../data/simpletransformer_lm_train.txt"
# test_file = "../data/simpletransformer_lm_test.txt"

In [108]:
use_cuda = torch.cuda.is_available()
model = LanguageModelingModel(
    "bert", "bert-base-uncased", args=model_args, use_cuda=use_cuda
)
print("Running on GPU: {}".format(use_cuda))

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another ar

Running on GPU: True


d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [109]:
model.train_model(train_file)

  0%|          | 0/6438 [00:19<?, ?it/s]


KeyboardInterrupt: 

In [110]:
model

In [111]:
model_type = 'bert'
model_name = 'bert-base-uncased'
rep_model1 = RepresentationModel(
        model_type=model_type,
        model_name=model_name,
        use_cuda=True
    )



sentences = ['hello world 1', 'hello world 2']
rep_model1.model = model

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [43]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [112]:
# encodings = tokenizer.encode(['hello world!'], truncation=True, padding='max_length', max_length=512)

# Encode the sentences.
encodings = tokenizer.encode_plus(
    text=['hello world1', 'hello world2'],  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 512,  # maximum length of a sentence
    padding='max_length',  # Add [PAD]s
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'pt',  # ask the function to return PyTorch tensors
)

In [113]:
encodings.to('cuda:0')
print(encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [114]:
output = model.model(**encodings)

In [115]:
output

MaskedLMOutput(loss=None, logits=tensor([[[ -6.3007,  -6.2235,  -6.2208,  ...,  -5.6089,  -5.4883,  -3.4899],
         [-11.8073, -12.0595, -11.9877,  ...,  -9.2797, -10.1229,  -6.2914],
         [-10.2800, -10.1213, -10.5442,  ...,  -8.4385, -10.0934,  -2.0034],
         ...,
         [ -5.0119,  -4.9243,  -5.0544,  ...,  -4.4180,  -5.7131,   0.1921],
         [ -5.1042,  -5.0389,  -5.1111,  ...,  -4.5710,  -5.5553,  -0.2378],
         [ -6.5133,  -6.4716,  -6.5902,  ...,  -5.8658,  -6.1577,  -1.3167]]],
       device='cuda:0', grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [94]:
def tokenize(text_list):
    # Tokenize the text with the provided tokenizer
    encoded = tokenizer.batch_encode_plus(
        text_list,
        add_special_tokens=True,
        max_length=512,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )
    return encoded

def batch_iterable(iterable, batch_size=1):
    l = len(iterable)
    for ndx in range(0, l, batch_size):
        yield iterable[ndx : min(ndx + batch_size, l)]

def encode_sentences(text_list, model, device='cpu'):
    """
    Generates list of contextual word or sentence embeddings using the model passed to class constructor
    :param text_list: list of text sentences
    :param combine_strategy: strategy for combining word vectors, supported values: None, "mean", "concat",
    or an int value to select a specific embedding (e.g. 0 for [CLS] or -1 for the last one)
    :param batch_size
    :return: list of lists of sentence embeddings (if `combine_strategy=None`) OR list of sentence
    embeddings (if `combine_strategy!=None`)
    """


    # model.to(device)
    # model.eval()
    embeddings = list()
    batches = batch_iterable(text_list, batch_size=32)
    for batch in batches:
        encoded = tokenize(batch)
        with torch.no_grad():
            
            token_vectors = model(
                input_ids=encoded["input_ids"].to(device),
                attention_mask=encoded["attention_mask"].to(device),
                token_type_ids=encoded["token_type_ids"].to(device),
            )
            # else:
            #     token_vectors = self.model(
            #         input_ids=encoded["input_ids"].to(self.device),
            #         attention_mask=encoded["attention_mask"].to(self.device),
            #     )
            return token_vectors
        embeddings.append(token_vectors.detach().numpy())
    embeddings = np.concatenate(embeddings, axis=0)

    return embeddings

In [95]:
temp = encode_sentences(['hello world1'], model.model, 'cuda:0')

In [96]:
temp

MaskedLMOutput(loss=None, logits=tensor([[[ -6.4943,  -6.4595,  -6.3881,  ...,  -5.6625,  -5.5103,  -4.1114],
         [ -9.5738,  -9.3839,  -9.1378,  ...,  -9.6869,  -7.1647,  -7.2835],
         [ -9.0334,  -9.3464,  -9.1769,  ...,  -9.1942,  -7.9128,  -4.7101],
         [ -9.0591,  -9.2856,  -9.5194,  ...,  -9.2929,  -8.7199,  -5.6519],
         [-12.1725, -12.0755, -11.7472,  ...,  -8.9967,  -9.7611,  -8.5034]]],
       device='cuda:0'), hidden_states=None, attentions=None)